In [ ]:
from textgenrnn import textgenrnn
from datetime import datetime
import os

In [ ]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 50,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
file_name = "wa2.txt"
model_name = 'WA'   # change to set file name of resulting trained models/texts

In [ ]:
# Code for importing model from files
textgen = textgenrnn(weights_path='WA_weights.hdf5',
                       vocab_path='WA_vocab.json',
                       config_path='WA_config.json')

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

In [ ]:
textgen.generate_samples(temperatures=[0.2, 0.5, 1.0, 1.2, 1.5])

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix_list = [
    "Koen Niemeijer",
    "Kevin Algera",
    "Willem Smits",
    "Gijs Cunnen",
    "Thijs van der Putten",
    "Wescel Manders",
    "Thomas Bardoel",
    "Michiel Arts",
    "Yoram Carboex"
]

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000

# Generate for everyone
gen_file = '{}_gentext_{}.txt'.format(model_name, "")
textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix="",
                         n=3000,
                         max_gen_length=max_gen_length)

# Generate for rest
for prefix in prefix_list:
    gen_file = '{}_gentext_{}.txt'.format(model_name, prefix)
    textgen.generate_to_file(gen_file,
                         temperature=temperature,
                         prefix=prefix,
                         n=n,
                         max_gen_length=max_gen_length)

